In [1]:
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline
from music21 import *
import os

In [2]:
# import midi file then convert to notes and create corpus

input_dir = 'midi_file'

corpus = list()

for mid in os.listdir(input_dir):
    # Load the MIDI file using music21
    midi_file = converter.parse(os.path.join(input_dir, mid))

    # Get a list of notes from the flattened stream
    # to convert to note with rest using list(midi_file.flatten().notesAndRests)
    # notes_list = [str(item) for item in list(midi_file.flat.notes) if isinstance(item, note.Note) or isinstance(item, chord.Chord) or isinstance(item, note.Unpitched)]
    notes_list = [str(item) for item in list(midi_file.flat.notes) if isinstance(item, note.Note) or isinstance(item, chord.Chord)]

    # append notes_list to corpus
    corpus.append(notes_list)

In [3]:
# desired_type = percussion.PercussionChord
# found_elements = []

# for element in list(midi_file.flat.notes):
#     if isinstance(element, desired_type):
#         found_elements.append(element)

# found_elements

In [4]:
# Preprocess the tokenized text for 3-grams language modelling
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, corpus)


from nltk.lm import MLE
trump_model = MLE(n) # Lets train a 3-grams model, previously we set n=3
trump_model.fit(train_data, padded_sents)

In [5]:
def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return content

In [6]:
music_notes = generate_sent(trump_model, num_words=1000, random_seed=42)

In [7]:
# create music21 object

music = list()

for melody in music_notes:
    if 'Note' in melody:
        # Extract the pitch from the string representation
        pitch_str = melody.split()[-1][:-1]

        # Create a music21.note.Note object
        note_obj = note.Note(pitch_str)

        music.append(note_obj)
        
    elif 'chord' in melody:
        # Remove the enclosing brackets and split the string into individual pitches
        pitch_strings = melody.replace("<music21.chord.Chord", "").replace(">", "").split()

        # Convert pitch strings to music21.pitch.Pitch objects
        pitch_objects = [chord.Chord(p).pitches[0] for p in pitch_strings]

        # Create a music21.chord.Chord object from the list of pitch objects
        chord_obj = chord.Chord(pitch_objects)

        music.append(chord_obj)
    
    # elif 'Unpitched' in melody:
    #     # Create an Unpitched object directly
    #     unpitched_object = note.Unpitched()

    #     music.append(unpitched_object)

In [8]:
# Create a new stream to hold the notes
reconstructed_stream = stream.Stream()

# Add notes to the new stream
for note in music:
    reconstructed_stream.append(note)

# Create a MIDI file from the reconstructed stream
output_midi_file = midi.translate.streamToMidiFile(reconstructed_stream)

# Save the output MIDI file
output_midi_file_path = 'output.mid'
output_midi_file.open(output_midi_file_path, 'wb')
output_midi_file.write()
output_midi_file.close()

print("MIDI file converted and saved as", output_midi_file_path)

MIDI file converted and saved as output.mid


In [9]:
len(music)

1000

In [10]:
music

[<music21.note.Note D>,
 <music21.chord.Chord A3 D4 F#3>,
 <music21.chord.Chord D3 A3 G4>,
 <music21.note.Note D>,
 <music21.chord.Chord F#4 D4 A4>,
 <music21.note.Note D>,
 <music21.chord.Chord A3 F#3 D3>,
 <music21.note.Note A>,
 <music21.note.Note D>,
 <music21.chord.Chord A3 C#4>,
 <music21.note.Note D>,
 <music21.chord.Chord F#3 A3 C#4>,
 <music21.chord.Chord A3 C#4>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note A>,
 <music21.chord.Chord D4 A3 G4>,
 <music21.note.Note A>,
 <music21.note.Note D>,
 <music21.chord.Chord A2 D3 A2 D3>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note A>,
 <music21.chord.Chord D4 F#4 A3 F#3 D3>,
 <music21.note.Note D>,
 <music21.chord.Chord A2 D4 F#3 D3 A3 F#4>,
 <music21.note.Note D>,
 <music21.note.Note A>,
 <music21.note.Note F#>,
 <music21.note.Note B>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note G>,
 <music21.note.Note B>,
 <music21.note.Note D>,
 <music21.n